In [23]:

import pandas as pd
import numpy as np
import numpy as np
import mglearn
from matplotlib import rc
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from lightgbm import LGBMRegressor

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# 나눔 폰트를 사용합니다.
rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

In [24]:
Sample = pd.read_excel('221021_Kaggle_Superstore.xls').iloc[:, 1:]

In [25]:
# 1.데이터 확인하기
# Sample.info()
    # null값이 없음
    # int : Postal Code, Sales, Quantity, Discount, Profit
# Sample.head()
Sample.columns


Index(['Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID',
       'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code',
       'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name',
       'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [26]:
# --- 가설 : 배송시간과 나라별, 카테고리별 관련성, 예측시간 구하기 -----
# -- 추가 : 배송시간이 길수록 재구매율이 떨어져서 이익율이 떨어진다 ----

# 2.필요없는 열 제거하기 (O)
# 3.배송까지 걸리는 일수를 계산해서 새로운 열(ShippingTime) 평균일수 구하기
# 3-1. Date 문자로 바꾸기
# 3-2. 인코딩하기
# 4.배송걸리는시간과 나라별, 카테고리별 관련성 조사(어떤 나라, 어떤 카테고리 제품이 오래걸리는가)
# 사전작업 : Oder Date는 0부터 시작해서 인코딩하기
# 5.배송걸리는 시간과 재구율 조사

Rebuy = Sample[[ 'Customer ID', 'Order ID', 'Order Date', 'Ship Date', 'Sub-Category', 'Sales', 'Quantity']]
# Rebuy = Sample

Rebuy['ShippingTime'] = Rebuy['Ship Date'] - Rebuy['Order Date']
Rebuy = Rebuy.drop(['Ship Date'], axis=1)
Rebuy = Rebuy[[ 'Customer ID', 'Order ID', 'Order Date', 'Sub-Category', 'Sales', 'Quantity','ShippingTime']]

Rebuy = Rebuy.sort_values(by='Order Date', ascending=True)
# Rebuy.sort_values(by=['Order Date'], ascending=False)
Rebuy['Order Date'] = Rebuy['Order Date'].apply(lambda x: str(x)[:10])
Rebuy['ShippingTime'] = Rebuy['ShippingTime'].apply(lambda x: int(str(x)[:1]))

# Rebuy['Order Date']
Rebuy

C:\Users\user\AppData\Local\Temp\ipykernel_21012\3741813226.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Rebuy['ShippingTime'] = Rebuy['Ship Date'] - Rebuy['Order Date']


,Customer ID,Order ID,Order Date,Sub-Category,Sales,Quantity,ShippingTime
7980,DP-13000,CA-2014-103800,2014-01-03,Paper,16.448,2,4
739,PO-19195,CA-2014-112326,2014-01-04,Labels,11.784,3,4
740,PO-19195,CA-2014-112326,2014-01-04,Storage,272.736,3,4
741,PO-19195,CA-2014-112326,2014-01-04,Binders,3.540,2,4
1759,MB-18085,CA-2014-141817,2014-01-05,Art,19.536,3,7
...,...,...,...,...,...,...,...
5091,JM-15580,CA-2017-156720,2017-12-30,Fasteners,3.024,3,4
908,PO-18865,CA-2017-143259,2017-12-30,Binders,52.776,3,4
907,PO-18865,CA-2017-143259,2017-12-30,Phones,90.930,7,4
1296,EB-13975,CA-2017-115427,2017-12-30,Binders,13.904,2,4


In [31]:
# np.mean(Rebuy['ShippingTime'])

def encode(series):
    dic = dict(zip(series.unique(), range(len(series.unique()))))
    result = series.apply(lambda x : dic[x])
    return result

change = ['Customer ID', 'Order ID', 'Order Date', 'Sub-Category']
for i in change:
# for i in Rebuy.columns:
    Rebuy[i] = encode(Rebuy[i])
    
# for i in Rebuy.columns:
# # for i in Rebuy.columns:
#     Rebuy[i] = encode(Rebuy[i])
    
# Rebuy.corrwith('ShippingTime')
Rebuy.corr()

,Customer ID,Order ID,Order Date,Sub-Category,Sales,Quantity,ShippingTime
Customer ID,1.000000,0.213346,0.226958,-0.005598,-0.002207,-0.007290,-0.016692
Order ID,0.213346,1.000000,0.995457,-0.001965,-0.009348,0.000525,-0.020523
Order Date,0.226958,0.995457,1.000000,-0.001800,-0.009495,0.000262,-0.021294
Sub-Category,-0.005598,-0.001965,-0.001800,1.000000,0.226957,-0.014537,-0.009114
Sales,-0.002207,-0.009348,-0.009495,0.226957,1.000000,0.200795,-0.007354
Quantity,-0.007290,0.000525,0.000262,-0.014537,0.200795,1.000000,0.018298
ShippingTime,-0.016692,-0.020523,-0.021294,-0.009114,-0.007354,0.018298,1.000000


In [29]:
Rebuy.head(3)

# 데이터 나누기(레이블과 feature)
X = Rebuy.iloc[:, 0:5].to_numpy()
y = Rebuy.iloc[:, 6].to_numpy()

# 훈련셋, 테스트셋 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

# 3.모델 선언
LR = LinearRegression()
KNN = KNeighborsRegressor(n_neighbors=10)
DT = DecisionTreeRegressor()
RF = RandomForestRegressor(n_estimators=50)
GBR = GradientBoostingRegressor(n_estimators=50)
XGB = xgb.XGBRegressor()
LGB = LGBMRegressor()

# 4.모델 학습
LR.fit(X_train, y_train)
KNN.fit(X_train, y_train)
DT.fit(X_train, y_train)
RF.fit(X_train, y_train)
GBR.fit(X_train, y_train)
XGB.fit(X_train, y_train)
LGB.fit(X_train, y_train)

# 5.테스트셋 넣기
predictLR = LR.predict(X_test)
predictKNN = KNN.predict(X_test)
predictDT = DT.predict(X_test)
predictRF = RF.predict(X_test)
predictGBR = GBR.predict(X_test)
predictXGB = XGB.predict(X_test)
predictLGB = LGB.predict(X_test)


# 6.모델 평가

print("===== 모델평가 =====")

scoreLR = r2_score(y_test, predictLR)
scoreKNN = r2_score(y_test, predictKNN)
scoreDT = r2_score(y_test, predictDT)
scoreRF = r2_score(y_test, predictRF)
scoreGBR = r2_score(y_test, predictGBR)
scoreXGB = r2_score(y_test, predictXGB)
scoreLGB = r2_score(y_test, predictLGB)

print("선형모델 : ", scoreLR)
print("KNN모델 : ", scoreKNN)
print("DT모델 : ", scoreDT)
print("RF모델 : ", scoreRF)
print("GBR모델 : ", scoreGBR)
print("XGB모델 : ", scoreXGB)
print("LGB모델 : ", scoreLGB)


===== 모델평가 =====
선형모델 :  -0.0002744002049928085
KNN모델 :  -0.058530576221242914
DT모델 :  -0.35258341355435774
RF모델 :  0.1772682120112672
GBR모델 :  0.01421621588366806
XGB모델 :  0.15287394585561465
LGB모델 :  0.03463221623410606


In [ ]:
Sample.columns
# Sample['Product Name'].unique() #1850
# Sample['Ship Mode'].unique()  #['Second Class', 'Standard Class', 'First Class', 'Same Day']
# Sample['Segment'].unique()  #['Consumer', 'Corporate', 'Home Office']
# Sample['Category'].unique()  #['Furniture', 'Office Supplies', 'Technology']
# Sample['Sub-Category'].unique() 
    #['Bookcases', 'Chairs', 'Labels', 'Tables', 'Storage', 'Furnishings', 'Art', 'Phones', 'Binders',
    #     'Appliances', 'Paper', 'Accessories', 'Envelopes', 'Fasteners', 'Supplies', 'Machines', 'Copiers']
# Sample['Discount'].unique() #[0.  , 0.45, 0.2 , 0.8 , 0.3 , 0.5 , 0.7 , 0.6 , 0.32, 0.1 , 0.4 ,0.15]
Sample['Sales'].unique() #[0.  , 0.45, 0.2 , 0.8 , 0.3 , 0.5 , 0.7 , 0.6 , 0.32, 0.1 , 0.4 ,0.15]
# 
 
# 'Order Date', 'Ship Date', 'Ship Mode'를 이용하여 재구매율에 대한 영향이 있나 확인 (실제로 관계가 있나?)
# Discount와 profit에 관계
# Sale와 Discount와 관계
Sample.head()
# Sample.corr()


,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
